In [39]:
from ttide.t_tide import t_tide
import ttide
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
## todo data1m into data2y(?)

In [2]:
#load dataset, Time(UTC),prs(m),rad(m)
df2 = pd.read_csv('data2y.csv',sep=',', header=0, index_col=0, squeeze=True, parse_dates=True)
#non-null object to float64
df2['prs(m)'] = pd.to_numeric(df2['prs(m)'], errors='coerce')
#prs(m) looks more stable
df2.drop(df2.columns[[1]], axis=1, inplace=True)

s = df2.interpolate()
s.head()

,prs(m)
Time(UTC),
2016-12-31 00:00:52,2.369
2016-12-31 00:01:53,2.360
2016-12-31 00:02:52,2.369
2016-12-31 00:03:53,2.361
2016-12-31 00:04:53,2.392


In [3]:
df2.index[0]

Timestamp('2016-12-31 00:00:52')

In [33]:
serie= s['prs(m)'] 
dt = 0.0166667
stime = df2.index[0]
lat = -33.02730833

In [42]:
1036988/60/24

720.1305555555556

In [34]:
out = t_tide(serie, dt, stime, lat)
#out = t_tide(dfp['prs(m) predictions'], dt=0.016, lat=np.array(-33.02730833))

/home/killgor/anaconda3/lib/python3.7/site-packages/ttide-0.3_exp-py3.7.egg/ttide/t_tide.py:345: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef = np.linalg.lstsq(lhs, rhs)[0].T
/home/killgor/anaconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:161: FutureWarning: `real` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.real(ser.to_numpy())` or `ser.to_numpy().real`.
  return val.real
/home/killgor/anaconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:204: FutureWarning: `imag` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.imag(ser.to_numpy())` or `ser.to_numpy().imag`.
  return val.imag


-----------------------------------
nobs = 1036988 
ngood = 1036987 
record length (days) = 720.13
start time: 2016-12-31 00:00:52
rayleigh criterion = 1.0

Greenwich phase computed with nodal
                   corrections applied to amplitude
                   and phase relative to center time

x0= 2.31  xtrend= 0
var(data)= 0.14    var(prediction)= 0.02    var(residual)= 0.13
var(prediction)/var(data) (%) = 14.3

        tidal amplitude and phase with 95 % CI estimates
 tide      freq        amp      amp_err   pha      pha_err    snr
* SA    0.0001141     0.0463     0.017    244.10     32.48       7.7
* SSA   0.0002282     0.0490     0.021    200.23     26.18       5.6
  MSM   0.0013098     0.0020     0.015    346.22    249.85     0.018
  MM    0.0015122     0.0091     0.018    233.60     90.63      0.25
  MSF   0.0028219     0.0019     0.014    338.50    173.16     0.018
  MF    0.0030501     0.0030     0.013    146.03    198.86     0.055
  ALP1  0.0343966     0.0008     0.017    

/home/killgor/anaconda3/lib/python3.7/site-packages/ttide-0.3_exp-py3.7.egg/ttide/t_utils.py:447: FutureWarning: `real` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.real(ser.to_numpy())` or `ser.to_numpy().real`.
  x = np.var(out['xingd'].real, ddof=1)
/home/killgor/anaconda3/lib/python3.7/site-packages/ttide-0.3_exp-py3.7.egg/ttide/t_utils.py:449: FutureWarning: `real` is deprecated and will be removed in a future version. To eliminate this warning for a Series `ser`, use `np.real(ser.to_numpy())` or `ser.to_numpy().real`.
  xr = np.var(out['xresgd'].real, ddof=1)


In [27]:
df2.index[-1]

Timestamp('2018-12-30 23:59:19')

In [28]:
out.keys()

dict_keys(['nobs', 'ngood', 'dt', 'xin', 'xout', 'xres', 'xingd', 'xoutgd', 'xresgd', 'isComplex', 'ray', 'nodcor', 'z0', 'dz0', 'fu', 'nameu', 'tidecon', 'snr', 'synth', 'lat', 'ltype', 'stime'])

In [ ]:
dates = read_csv('predictions.csv',sep=',', header=0, index_col=0, squeeze=True, parse_dates=True)

In [ ]:
#getting te timestamp

In [41]:
ttide.t_predic(df2.values, out.get("nameu"), out.get("fu"), out.get("tidecon"), lat)

array([nan, nan, nan, ..., nan, nan, nan])